In [8]:
import os 

import tensorflow as tf
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout
from keras.activations import relu, softmax
from keras.optimizers import Adam 
from keras.losses import categorical_crossentropy
from tensorflow.python.keras.utils.data_utils import get_file

import cv2


In [2]:
with open("/media/deborshiroy/MY DATA/My Projects/birds classification/app/coco.names", "r") as f:
    classesList = f.read().splitlines()    

In [3]:
classesList

['__Background__',
 'person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'street sign',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'hat',
 'backpack',
 'umbrella',
 'shoe',
 'eye glasses',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'plate',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'mirror',
 'dining table',
 'window',
 'desk',
 'toilet',
 'door',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'blender',
 'book',
 'cloc

In [21]:
LABELS = ["birds"]

PRETRAINED_MODEL_FOLDER = "pretrained_model"

EFFICIENTDET_D4_MODEL_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d4_coco17_tpu-32.tar.gz"
MODEL_NAME = EFFICIENTDET_D4_MODEL_URL.split("/")[-1].split(".")[0]
FILE_NAME = EFFICIENTDET_D4_MODEL_URL.split("/")[-1]



print(f"{MODEL_NAME} model is loading ")
tf.keras.backend.clear_session()
model = tf.saved_model.load(
os.path.join(
"../../pretrained_model",
"checkpoints",
MODEL_NAME,
"saved_model"
)
)

efficientdet_d4_coco17_tpu-32 model is loading 


2022-09-12 11:33:21.753576: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-12 11:33:21.761768: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-12 11:33:21.761980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-12 11:33:21.762448: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [28]:
import numpy as np
image = cv2.imread("../test_data/images (1).jpeg")
inputTensor = cv2.cvtColor(
    image, 
    cv2.COLOR_BGR2RGB
)
inputTensor = tf.convert_to_tensor(
    inputTensor,
    dtype=tf.uint8
)
inputTensor = inputTensor[tf.newaxis,...]

detections = model(inputTensor)
bboxes = detections["detection_boxes"][0].numpy()
classIndexes = detections["detection_classes"][0].numpy().astype(np.int32)
classScores = detections["detection_scores"][0].numpy()

imH, imW, imC = image.shape
bboxidx = tf.image.non_max_suppression(
    bboxes,
    classScores,
    max_output_size=50,
    iou_threshold=0.5,
    score_threshold=0.5
)
print(bboxidx)
if len(bboxidx) != 0:
    for i in bboxidx:
        bbox = tuple(bboxes[i].tolist())
        classConfidence = round(100*classScores[i])
        classIndex = classIndexes[i]

        classLabelText = LABELS[0]
        #classColor = colorsList[classIndex]

        displayText = f"{classLabelText}: {classConfidence}%"

        ymin, xmin, ymax, xmax = bbox

        xmin, xmax, ymin, ymax = (xmin * imW, xmax * imW, ymin * imH, ymax * imH)
        xmin, xmax, ymin, ymax = int(xmin), int(xmax), int(ymin), int(ymax)

        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color = (255, 0, 0), thickness=1)
        cv2.putText(image, displayText, (xmin, ymin-10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2) 

cv2.imshow("Result", image)
cv2.waitKey(0)
cv2.destroyAllWindows()



tf.Tensor([0], shape=(1,), dtype=int32)


In [5]:
def predictImage(self, imagePath, threshold=0.5):
    image = cv2.imread(imagePath)
    bboximage = self.createBoundingBox(image, threshold)
    #cv2.imwrite(self.modelName+"jpg", bboximage)

    cv2.imshow("Result", bboximage)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [10]:
predictImage("app/test_data/images (1).jpeg", threshold=0.5)

TypeError: predictImage() missing 1 required positional argument: 'imagePath'